# Data Consistency in Distributed Systems

Understanding how to maintain data consistency across distributed systems is fundamental to system design. This notebook covers the theoretical foundations and practical patterns for handling consistency challenges.

## Topics Covered
- CAP Theorem Deep Dive
- ACID vs BASE Properties
- Consistency Levels (Strong, Eventual, Causal)
- Distributed Transactions (2PC, Saga Pattern)
- Conflict Resolution Strategies
- Vector Clocks & CRDTs

## 1. CAP Theorem Deep Dive

The **CAP Theorem** (Brewer's Theorem) states that a distributed system can provide at most **two out of three** guarantees simultaneously:

```
                    ┌─────────────────┐
                    │   Consistency   │
                    │  (All nodes see │
                    │   same data)    │
                    └────────┬────────┘
                             │
              ┌──────────────┼──────────────┐
              │              │              │
              ▼              │              ▼
    ┌─────────────────┐      │    ┌─────────────────┐
    │  Availability   │◄─────┴───►│   Partition     │
    │ (Every request  │           │   Tolerance     │
    │  gets response) │           │ (System works   │
    └─────────────────┘           │  despite splits)│
                                  └─────────────────┘
```

### CAP Trade-off Choices

| Choice | Description | Examples |
|--------|-------------|----------|
| **CP** | Consistent + Partition Tolerant | MongoDB, HBase, Redis Cluster, Zookeeper |
| **AP** | Available + Partition Tolerant | Cassandra, DynamoDB, CouchDB, Riak |
| **CA** | Consistent + Available | Traditional RDBMS (single node), not practical for distributed |

### Key Insight: PACELC Extension
```
If (Partition) → choose Availability or Consistency
Else           → choose Latency or Consistency

┌──────────────────────────────────────────────────────┐
│  PACELC: PA/EL (DynamoDB)  vs  PC/EC (MongoDB)       │
│                                                      │
│  During Partition:  Availability vs Consistency      │
│  Else (Normal):     Latency vs Consistency           │
└──────────────────────────────────────────────────────┘
```

## 2. ACID vs BASE Properties

### ACID (Traditional Databases)
```
┌────────────────────────────────────────────────────────────────┐
│                         ACID                                   │
├────────────────┬───────────────────────────────────────────────┤
│ Atomicity      │ All or nothing - transaction fully completes │
│                │ or fully rolls back                          │
├────────────────┼───────────────────────────────────────────────┤
│ Consistency    │ Database moves from one valid state to       │
│                │ another valid state                          │
├────────────────┼───────────────────────────────────────────────┤
│ Isolation      │ Concurrent transactions don't interfere      │
│                │ (as if executed serially)                    │
├────────────────┼───────────────────────────────────────────────┤
│ Durability     │ Committed data survives system failures      │
└────────────────┴───────────────────────────────────────────────┘
```

### BASE (Distributed NoSQL)
```
┌────────────────────────────────────────────────────────────────┐
│                         BASE                                   │
├────────────────┬───────────────────────────────────────────────┤
│ Basically      │ System guarantees availability (CAP's A)     │
│ Available      │                                              │
├────────────────┼───────────────────────────────────────────────┤
│ Soft State     │ State may change over time, even without     │
│                │ input (due to eventual consistency)          │
├────────────────┼───────────────────────────────────────────────┤
│ Eventually     │ System will become consistent over time      │
│ Consistent     │ given no new updates                         │
└────────────────┴───────────────────────────────────────────────┘
```

### Comparison Matrix
| Aspect | ACID | BASE |
|--------|------|------|
| Consistency | Strong, immediate | Eventual, relaxed |
| Availability | May sacrifice for consistency | Prioritized |
| Scalability | Harder to scale horizontally | Designed for scale |
| Use Case | Banking, inventory | Social feeds, analytics |

## 3. Consistency Levels

### Spectrum of Consistency Models
```
Strong ◄──────────────────────────────────────────────► Weak
  │                                                       │
  │   Linearizable → Sequential → Causal → Eventual       │
  │        │             │           │          │         │
  │    Real-time     Program     Cause &    Eventually    │
  │     ordering      order      effect     converges     │
  │                                                       │
  └─── More Coordination ───────── Less Coordination ─────┘
```

### Strong Consistency (Linearizability)
- Every read returns the most recent write
- Appears as a single copy of data
- **Cost**: Higher latency, lower availability

```
Client A: Write(x=1) ─────────┐
                              ▼
                    ┌─────────────────┐
                    │   Coordinator   │──► Replicate to ALL
                    │   (blocks)      │◄── Wait for ALL ACKs
                    └─────────────────┘
                              │
Client B: Read(x) ────────────┴──► Returns x=1 (guaranteed)
```

### Eventual Consistency
- Given enough time (no new writes), all replicas converge
- May read stale data temporarily
- **Benefit**: High availability, low latency

```
Time ────────────────────────────────────────────────────►

Node A: [x=1] ─────────────────────────► [x=2]
              \                         /
               \   (propagation)       /
                \                     /
Node B: [x=0] ──► [x=1] ────────────► [x=2]  ← Eventually same!
```

### Causal Consistency
- Preserves cause-and-effect relationships
- If A causes B, everyone sees A before B
- Concurrent operations may be seen in different orders

```
User A: Post("Hello")  ─────causes────► Reply("Hi back!")
            │                                  │
            ▼                                  ▼
All users MUST see "Hello" before seeing "Hi back!"
(But unrelated posts can appear in any order)
```

## 4. Distributed Transactions

### Two-Phase Commit (2PC)
Ensures atomic commits across multiple nodes.

```
┌─────────────────────────────────────────────────────────────────┐
│                     PHASE 1: PREPARE                            │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   Coordinator                                                   │
│       │                                                         │
│       ├──────── PREPARE? ────────► Participant A ──► VOTE YES  │
│       │                                                         │
│       ├──────── PREPARE? ────────► Participant B ──► VOTE YES  │
│       │                                                         │
│       └──────── PREPARE? ────────► Participant C ──► VOTE YES  │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│                     PHASE 2: COMMIT                             │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   Coordinator (all voted YES)                                   │
│       │                                                         │
│       ├──────── COMMIT! ─────────► Participant A ──► ACK       │
│       ├──────── COMMIT! ─────────► Participant B ──► ACK       │
│       └──────── COMMIT! ─────────► Participant C ──► ACK       │
│                                                                 │
│   (If ANY voted NO → send ROLLBACK to all)                      │
└─────────────────────────────────────────────────────────────────┘
```

**2PC Limitations**: Blocking protocol, coordinator is single point of failure, high latency.

---

### Saga Pattern
Sequence of local transactions with compensating actions for rollback.

```
┌─────────────────────────────────────────────────────────────────┐
│                    SAGA: Order Checkout                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  T1: Create Order ──► T2: Reserve Stock ──► T3: Charge Payment  │
│         │                    │                     │            │
│         │                    │                     X (FAILS!)   │
│         │                    │                     │            │
│         ▼                    ▼                     ▼            │
│  C1: Cancel Order ◄── C2: Release Stock ◄── Compensate chain   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘

Choreography (Event-driven):     Orchestration (Central control):
┌────┐    ┌────┐    ┌────┐       ┌──────────────┐
│ S1 │───►│ S2 │───►│ S3 │       │ Orchestrator │
└────┘    └────┘    └────┘       └──────┬───────┘
  Events flow between services         │ Commands to each
                                  ┌────┼────┐
                                  ▼    ▼    ▼
                                 S1   S2   S3
```

| Aspect | 2PC | Saga |
|--------|-----|------|
| Atomicity | Strong (all-or-nothing) | Eventual (compensating) |
| Blocking | Yes | No |
| Isolation | Full | Requires careful design |
| Scale | Limited | Better for microservices |

## 5. Conflict Resolution Strategies

When concurrent writes occur, conflicts must be resolved:

### Last-Write-Wins (LWW)
```
Node A: x=1 (t=100) ──┐
                      ├──► Conflict! ──► x=2 wins (t=105 > t=100)
Node B: x=2 (t=105) ──┘

⚠️ Risk: Data loss if timestamps are skewed
```

### Application-Level Merge
```
Shopping Cart Example:
┌───────────────────────────────────────────────────────┐
│  Replica A: {item: "book", qty: 2}                    │
│  Replica B: {item: "book", qty: 3}                    │
│                    │                                  │
│                    ▼                                  │
│  Merge Strategy: MAX(qty) → {item: "book", qty: 3}   │
│  (or: SUM, UNION for sets, custom logic)              │
└───────────────────────────────────────────────────────┘
```

### Multi-Version Concurrency Control (MVCC)
```
┌─────────────────────────────────────────────────────────┐
│ Version Chain for key 'x':                              │
│                                                         │
│  v1(t=100) ──► v2(t=110) ──► v3(t=120)                 │
│    x=5          x=10          x=15                      │
│                                                         │
│ Reader at t=115 sees v2 (x=10)                          │
│ Reader at t=125 sees v3 (x=15)                          │
└─────────────────────────────────────────────────────────┘
```

### Resolution Strategy Selection
| Strategy | Use When | Examples |
|----------|----------|----------|
| LWW | Order doesn't matter, latest wins | Session data, cache |
| Merge | Conflicts can be combined logically | Counters, sets |
| Manual | User must decide | Document editing |
| CRDTs | Automatic conflict-free merge | Collaborative apps |

## 6. Vector Clocks

Vector clocks track causality in distributed systems by maintaining a vector of logical timestamps.

```
┌─────────────────────────────────────────────────────────────────┐
│  Vector Clock: [NodeA: 2, NodeB: 3, NodeC: 1]                   │
│                                                                 │
│  Each node increments its own counter on local events          │
│  Vectors are merged (take max) when messages are received      │
└─────────────────────────────────────────────────────────────────┘

Example: Three nodes tracking writes to key 'x'

Time ──────────────────────────────────────────────────────────►

Node A: [1,0,0]─write(x=5)──►[2,0,0]────────────────────────────
            │                    │
            └──── sync ──────────┼────────────────────┐
                                 │                    │
Node B: [0,0,0]─────────────►[2,1,0]─write(x=7)──►[2,2,0]
                                                      │
                                                      │
Node C: [0,0,0]───write(x=9)──►[0,0,1]────────────────┘
                                   │                  │
                                   └─────sync─────────┘
                                          │
                                          ▼
                              ┌───────────────────────┐
                              │ CONFLICT DETECTED!    │
                              │ [2,2,0] vs [0,0,1]    │
                              │ Neither dominates     │
                              │ (concurrent writes)   │
                              └───────────────────────┘
```

### Comparing Vector Clocks
```
V1 = [2, 3, 1]    V2 = [2, 4, 1]

V1 < V2?  Check: all(V1[i] <= V2[i]) AND any(V1[i] < V2[i])
          [2<=2, 3<=4, 1<=1] = True, and 3<4 → V1 < V2 (V2 is newer)

V3 = [3, 2, 1]    V4 = [2, 3, 1]

V3 vs V4?  3>2 but 2<3 → CONCURRENT (conflict!)
```

### Limitations
- Vector size grows with number of nodes (use version vectors or dotted version vectors)
- Need mechanism to prune/garbage collect old entries

## 7. CRDTs (Conflict-free Replicated Data Types)

Data structures that can be replicated across nodes and **always merge without conflicts**.

### Core Property
```
┌─────────────────────────────────────────────────────────────────┐
│  CRDT Guarantee: Replicas can diverge, but will ALWAYS         │
│  converge to the same state when merged - NO COORDINATION!     │
└─────────────────────────────────────────────────────────────────┘
```

### Common CRDT Types

**G-Counter (Grow-only Counter)**
```
Node A: {A:5, B:0, C:0} ─┐
Node B: {A:0, B:3, C:0} ─┼──► Merge: {A:5, B:3, C:2} = 10
Node C: {A:0, B:0, C:2} ─┘    (take MAX per node, then SUM)
```

**PN-Counter (Increment/Decrement)**
```
┌─────────────────────────────────────────┐
│  P (positive): G-Counter for increments │
│  N (negative): G-Counter for decrements │
│  Value = sum(P) - sum(N)                │
└─────────────────────────────────────────┘
```

**LWW-Register (Last-Writer-Wins Register)**
```
Register = (value, timestamp)
Merge: keep entry with highest timestamp
```

**OR-Set (Observed-Remove Set)**
```
Add(x) → attach unique tag
Remove(x) → remove all observed tags for x

Node A: add(x) ────────────► {x: [tag1]}
Node B: add(x), remove(x) ─► {x: [tag2]} → {} (tag2 removed)
                                   │
                                   ▼
Merge: {x: [tag1]} ∪ {} = {x: [tag1]}  ← x survives!
(add wins over concurrent remove)
```

### CRDT Use Cases
| CRDT Type | Use Case |
|-----------|----------|
| G-Counter | Page views, likes |
| PN-Counter | Inventory, upvotes/downvotes |
| LWW-Register | User profile, last-seen |
| OR-Set | Shopping cart, tags |
| LWW-Map | Distributed config, preferences |

## 8. Summary & Decision Framework

```
┌─────────────────────────────────────────────────────────────────┐
│              CHOOSING A CONSISTENCY APPROACH                    │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Need ACID guarantees? ──YES──► Traditional DB (PostgreSQL)    │
│         │                       + 2PC for distributed          │
│         NO                                                      │
│         ▼                                                       │
│  High availability critical? ──YES──► BASE + Eventual          │
│         │                             (Cassandra, DynamoDB)    │
│         NO                                                      │
│         ▼                                                       │
│  Microservices transactions? ──YES──► Saga Pattern             │
│         │                                                       │
│         NO                                                      │
│         ▼                                                       │
│  Conflict-free updates needed? ──YES──► CRDTs                  │
│         │                                                       │
│         NO                                                      │
│         ▼                                                       │
│  Use Causal Consistency with Vector Clocks                      │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Key Takeaways

| Concept | Remember |
|---------|----------|
| CAP Theorem | Pick 2: Consistency, Availability, Partition Tolerance |
| ACID vs BASE | Strong consistency vs. High availability trade-off |
| Strong Consistency | All reads see latest write (higher latency) |
| Eventual Consistency | Replicas converge over time (lower latency) |
| 2PC | Atomic but blocking - use for critical transactions |
| Saga | Non-blocking with compensations - use for microservices |
| Vector Clocks | Detect causality and concurrent updates |
| CRDTs | Automatic conflict resolution - great for collaboration |

### Further Reading
- [Designing Data-Intensive Applications](https://dataintensive.net/) - Martin Kleppmann
- [CAP Twelve Years Later](https://www.infoq.com/articles/cap-twelve-years-later-how-the-rules-have-changed/) - Eric Brewer
- [CRDTs: Consistency without concurrency control](https://arxiv.org/abs/0907.0929)

# Consistency Models

- Strong vs eventual consistency
- CAP/PACELC
- Quorums and tunable consistency
